# Multiple model training , pipeline and column transformer

In [102]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [103]:
data = sns.load_dataset('tips')
data.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


***predict whats the time? if its Lunch or dinner***


In [104]:
# predict whats the time? if its Lunch or dinner
data['time'].unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [105]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


***# Label Encoder***

In [106]:
from sklearn.preprocessing import LabelEncoder

Encoder = LabelEncoder()
data['time'] = Encoder.fit_transform(data['time'])

In [107]:
data.head(2)   # lunch = 0 , Dinner = 1

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,0,2
1,10.34,1.66,Male,No,Sun,0,3


In [108]:
X = data.drop('time' , axis=1)
y = data['time']

In [109]:
X , y

(     total_bill   tip     sex smoker   day  size
 0         16.99  1.01  Female     No   Sun     2
 1         10.34  1.66    Male     No   Sun     3
 2         21.01  3.50    Male     No   Sun     3
 3         23.68  3.31    Male     No   Sun     2
 4         24.59  3.61  Female     No   Sun     4
 ..          ...   ...     ...    ...   ...   ...
 239       29.03  5.92    Male     No   Sat     3
 240       27.18  2.00  Female    Yes   Sat     2
 241       22.67  2.00    Male    Yes   Sat     2
 242       17.82  1.75    Male     No   Sat     2
 243       18.78  3.00  Female     No  Thur     2
 
 [244 rows x 6 columns],
 0      0
 1      0
 2      0
 3      0
 4      0
       ..
 239    0
 240    0
 241    0
 242    0
 243    0
 Name: time, Length: 244, dtype: int32)

In [110]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X , y , test_size = 0.3 , random_state = 0)

In [111]:
X_train.head(5)

,total_bill,tip,sex,smoker,day,size
61,13.81,2.00,Male,Yes,Sat,2
146,18.64,1.36,Female,No,Thur,3
52,34.81,5.20,Female,No,Sun,4
66,16.45,2.47,Female,No,Sat,2
26,13.37,2.00,Male,No,Sat,2


In [112]:
data.isna().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

# ColumnTransformer

***# feature engineering automation using pipeline and columntransformer***

In [113]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


In [114]:
cat_col = ['sex','smoker','day']
num_col = ['total_bill' , 'tip' , 'size']

In [115]:
Pipeline_num = Pipeline(steps = [('Imputers' , SimpleImputer(strategy = 'median')) , ('Scaler' , StandardScaler())])

In [116]:
Pipeline_cat = Pipeline(steps = [('Imputers' , SimpleImputer(strategy = 'most_frequent')) , ('Encoder' , OneHotEncoder())])

In [117]:
preprocessor = ColumnTransformer([('Pipeline_num' , Pipeline_num , num_col) , ('Pipeline_cat' , Pipeline_cat , cat_col)])

In [118]:
X_train = preprocessor.fit_transform(X_train)

X_test = preprocessor.transform(X_test)

In [119]:
X_train

array([[-0.68232142, -0.68502117, -0.6164355 , ...,  1.        ,
         0.        ,  0.        ],
       [-0.15257538, -1.13966829,  0.46391538, ...,  0.        ,
         0.        ,  1.        ],
       [ 1.62092224,  1.58821446,  1.54426626, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [-1.02890475, -1.04021424, -0.6164355 , ...,  0.        ,
         0.        ,  1.        ],
       [ 1.35659761,  2.15652336,  1.54426626, ...,  0.        ,
         1.        ,  0.        ],
       [-1.40181086,  1.55269515, -0.6164355 , ...,  0.        ,
         1.        ,  0.        ]])

In [120]:
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [121]:
models = {
    'Support vector classifier' : SVC(),
    'Decision Tree Classifier' : DecisionTreeClassifier()
}

In [132]:
from sklearn.metrics import accuracy_score

def model_train_eval(X_train, y_train, X_test, y_test, models):
    evaluation = {}
    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        model_score = accuracy_score(y_test, y_pred)*100
        evaluation[name] = model_score
    return evaluation


In [133]:
# Assume X_train, y_train, X_test, y_test are predefined
evaluation_results = model_train_eval(X_train, y_train, X_test, y_test, models)

evaluation_results

{'Support vector classifier': 91.8918918918919,
 'Decision Tree Classifier': 93.24324324324324}